In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

USER = 'aacuser'
PASS = 'SNHU1234'
HOST = 'nv-desktop-services.apporto.com'
PORT = 32448
DB = 'AAC'
COL = 'animals'
db = AnimalShelter(USER, PASS, HOST, PORT, DB, COL)

# Connect to database via CRUD Module


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)

if '_id' in df.columns:
    df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard - Juan Rodriguez'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
            style={'height': '200px', 'width':'auto'}
            ),
    html.Hr(),
    html.Div([
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water'},
            {'label': 'Mountain or Wilderness Rescue', 'value':'Mountain'},
            {'label': 'Disaster or Individual Tracking', 'value':'Disaster'},
            {'label': 'Reset', 'value':'Reset'}
        ]
        
        )
    ]
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
        row_selectable ="single",
        page_size= 10,
        page_current=0,
        selected_rows=[],
        page_action="native",
        filter_action="native",
        sort_action="native",
        
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
              Output('datatable-id','columns'),
              Output('datatable-id','selected_rows')],
                     
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    if filter_type == 'Reset':
        df = pd.DataFrame.from_records(db.read({}))
    #water rescue filter
    elif filter_type == 'Water':
        df = pd.DataFrame.from_records(list(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"
                             ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0},
            "age_upon_outcome_in_weeks":{"$lte":156.0},
            "outcome_type":{"$nin" :["Euthanasia"]}
        })))
    elif filter_type == 'Mountain':
    #mountain rescue filter
        df = pd.DataFrame.from_records(list(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog",
                              "Siberian Husky", "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0},
            "age_upon_outcome_in_weeks":{"$lte":156.0},
            "outcome_type":{"$nin": ["Euthanasia"]}
    })))
    elif filter_type == 'Disaster':
    #disaster rescue filter
        df = pd.DataFrame.from_records(list(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever",
                              "Bloodhound", "Rottweiler"
                             ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0},
            "age_upon_outcome_in_weeks":{"$lte":300.0},
            "outcome_type":{"$nin": ["Euthanasia"]}
    })))
    #reset no filter
    else:
        df = pd.DataFrame.from_records(animals.read({}))
    
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data = df.to_dict('records')
    selected_rows = [0]
    
    return (data,columns, selected_rows)
    

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return [dcc.Graph(figure={})] #Return an empty fig if there is no data
    
    dff = pd.DataFrame.from_dict(viewData)
    
    print ("Filtered Data:")
    print(dff.head())
    df_grouped = dff.groupby(['breed']).size().reset_index(name='Count')
    
    fig = px.pie(df_grouped, values='Count', names='breed', title="Distribution of Animal Breeds")
    
    return [
        dcc.Graph(figure=fig)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:23189/
Filtered Data:
   rec_num age_upon_outcome animal_id animal_type                     breed  \
0        1          3 years   A746874         Cat    Domestic Shorthair Mix   
1        2           1 year   A725717         Cat    Domestic Shorthair Mix   
2        3          2 years   A716330         Dog   Chihuahua Shorthair Mix   
3        5          2 years   A691584         Dog    Labrador Retriever Mix   
4        6          5 years   A696004         Dog  Cardigan Welsh Corgi Mix   

          color date_of_birth             datetime            monthyear  \
0   Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1  Silver Tabby    2015-05-02  2016-05-06 10:49:00  2016-05-06T10:49:00   
2   Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
3     Tan/White    2012-11-06  2015-05-30 13:48:00  2015-05-30T13:48:00   
4   Sable/White    2010-01-27  2015-01-28 10:39:00  2015-01-28T10:39:00   

    name outcom

Filtered Data:
   rec_num age_upon_outcome animal_id animal_type                   breed  \
0      327         2 months   A759505         Dog  Labrador Retriever Mix   
1      381          1 month   A736066         Dog  Labrador Retriever Mix   
2      699         5 months   A709048         Dog  Labrador Retriever Mix   
3      732          2 years   A749782         Dog  Labrador Retriever Mix   
4     1121           1 year   A757158         Dog  Labrador Retriever Mix   

         color date_of_birth             datetime            monthyear  \
0        White    2017-08-02  2017-10-04 15:42:00  2017-10-04T15:42:00   
1          Tan    2016-08-03  2016-10-03 17:17:00  2016-10-03T17:17:00   
2  Black/White    2015-02-08  2015-08-08 17:58:00  2015-08-08T17:58:00   
3    Tan/White    2015-05-19  2017-07-25 14:59:00  2017-07-25T14:59:00   
4  White/Black    2016-08-30  2017-08-31 14:12:00  2017-08-31T14:12:00   

        name outcome_subtype     outcome_type sex_upon_outcome  location_lat 

Filtered Data:
   rec_num age_upon_outcome animal_id animal_type                   breed  \
0      327         2 months   A759505         Dog  Labrador Retriever Mix   
1      381          1 month   A736066         Dog  Labrador Retriever Mix   
2      699         5 months   A709048         Dog  Labrador Retriever Mix   
3      732          2 years   A749782         Dog  Labrador Retriever Mix   
4     1121           1 year   A757158         Dog  Labrador Retriever Mix   

         color date_of_birth             datetime            monthyear  \
0        White    2017-08-02  2017-10-04 15:42:00  2017-10-04T15:42:00   
1          Tan    2016-08-03  2016-10-03 17:17:00  2016-10-03T17:17:00   
2  Black/White    2015-02-08  2015-08-08 17:58:00  2015-08-08T17:58:00   
3    Tan/White    2015-05-19  2017-07-25 14:59:00  2017-07-25T14:59:00   
4  White/Black    2016-08-30  2017-08-31 14:12:00  2017-08-31T14:12:00   

        name outcome_subtype     outcome_type sex_upon_outcome  location_lat 